# Attention Vizualizer

Howdy! Regardless of what shows up below, click Kernel --> Restart and Run All to get started.

Mouse over each Glimpse to view the post-windowing input to the network. It might be kind of slow, since it's a network round trip, so be patient. Enjoy!

In [1]:
from viz import *

Widget Javascript not detected.  It may not be installed or enabled properly.


W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='271ed139-5f63-4cc5-a62f-93100de9b62f', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='375ccc1d-d863-4b38-ae36-3e8dfdad57d4', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='3c1c4f97-a767-4459-b72e-bc700228c59d', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='7a987481-43d8-4b93-93d5-9868bfc2db56', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='7ae2894e-55a6-4c46-96f6-c2e69219eafe', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='8273838a-add4-418d-9c7f-fb372c22e235', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='849c4227-df00-44ab-b0e3-32286f1da873', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='90f116cc-5238-4917-a287-5301609f5808', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='99f848c4-0f36-44c3-8f10-74aa93ae7b0a', ...)
W

ValueError: cannot reshape array of size 100 into shape (1,100,100)

In [ ]:
from analysis import classify_imgs2

In [ ]:
# import numpy as np
# import scipy.special

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from random import randint

from scipy import linspace
from scipy import pi,sqrt,exp
from scipy.special import erf
import scipy

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show


clear_output()
b2 = Button(description="Click to Start", icon="arrow", width=400)

dropdown2 = Dropdown(options=['0', '1000', '2000', '3000', '4000', '5000', '10000', '20000', '30000', '40000', '50000',
                             '60000', '70000', '80000', '90000', '100000', '110000', '120000', '130000', '140000', '150000',
                             '160000', '170000', '180000', '190000', '200000', '250000', '300000', '400000', '500000',
                             '600000', '700000', '800000', '900000', '910000', '920000', '1000000', '1100000', '1200000', '1300000', '1400000', '1500000'],
                    value='10000', 
                    description='Iteration:'
)

data = None
    
    
def get_pdf(sigma, x, mu):
    """Calculate the pdf."""
    
#     pdf = 1/(x* sigma * np.sqrt(2*np.pi)) * np.exp(-(np.log(x)-mu)**2 / (2*sigma**2))
    pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
    return pdf


def get_cdf(sigma, x, mu, a):
    """Calculate the cdf."""
    
#     cdf = (1 + erf(a*x / sqrt(2))) / 2 
    cdf = (1 + scipy.special.erf((a*x - mu) / np.sqrt(2*sigma**2))) / 2
    return cdf


def get_p(pdf, cdf):
    """Calculate p to create skew curve."""
    
    p = 2 / pdf * cdf / 100000
    return p

    
def curve(sigma, x, mu, plot, label="", color="gray"):
    """Add normal distribution curve to plot."""
    
    pdf = get_pdf(sigma, x, mu)
    plot.line(x, pdf, line_color=color, line_width=8, alpha=1, legend=label)
    
    
def skew_curve(sigma, x, mu, a, plot):
    """Add a skewed curve to the plot."""
    
    p = get_p(get_pdf(sigma, x, mu), get_cdf(sigma, x, mu, a))

    def f(x): return 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
    max_x = scipy.optimize.fmin(lambda x: -f(x), a)

    plot.line(9-x, p / f(max_x), line_color="blue", line_width=8, alpha=0.4)
    
    
def combined_curve(sigma, x, mu, a, plot):
    """Add curve combining skew and normal distribution to plot."""
    
    p = get_p(get_pdf(sigma, x, mu), get_cdf(sigma, x, mu, a))
    
    def f(x): return 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
    max_x = scipy.optimize.fmin(lambda x: -f(x), a)

    def f2(x): return (1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2)) + p / f(max_x))
    max_x2 = scipy.optimize.fmin(lambda x: -f(x), a)

    plot.line(x, (((1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2)))
            + p / f(max_x))/ f2(max_x2)), line_color="purple", line_width=2, alpha=1)
    

def update_curves():
    clear_output()
    global data
    num_imgs = 10000
    print("number of images: %d" % num_imgs)
    imgs_data = classify_imgs2(int(dropdown2.value), True, num_imgs)
    
#     num_blobs = randint(0, 9)
    max_blobs = 9
    min_blobs = 0
    
    curves = list()
    dark = "#111111"
    light = "#DDDDDD"
    p2 = figure(title="Blob Number Classification Probabilities Distributions", y_range=(0, 1), tools="save", background_fill_color=dark)
    p2.border_fill_color = dark
    p2.title.text_color = light
    p2.xaxis.axis_label_text_color = light
    p2.yaxis.axis_label_text_color = light
    p2.xaxis.axis_line_color = light
    p2.yaxis.axis_line_color = light
    p2.xaxis.major_label_text_color = light
    p2.yaxis.major_label_text_color = light
    p2.xaxis.major_tick_line_color = light
    p2.xaxis.minor_tick_line_color = light
    p2.yaxis.major_tick_line_color = light
    p2.yaxis.minor_tick_line_color = light

    for num_blobs in range(min_blobs, max_blobs + 1):
        print("num_blobs: ", num_blobs)
        

        p1 = figure(title="Blob Number Classification Probabilities Distribution for %d Blobs" % num_blobs, y_range=(0, 1), tools="save",
                    background_fill_color=dark)
        p1.border_fill_color = dark
        p1.title.text_color = light
        p1.xaxis.axis_label_text_color = light
        p1.yaxis.axis_label_text_color = light
        p1.xaxis.axis_line_color = light
        p1.yaxis.axis_line_color = light
        p1.xaxis.major_label_text_color = light
        p1.yaxis.major_label_text_color = light
        p1.xaxis.major_tick_line_color = light
        p1.xaxis.minor_tick_line_color = light
        p1.yaxis.major_tick_line_color = light
        p1.yaxis.minor_tick_line_color = light



        m = 0.1
        z_size = 10

        new_hist = np.zeros(z_size)
        choice_hist = np.zeros(z_size)
        value_counts = np.zeros(z_size)
        values_sum = 0
        sqr_sum = 0
        num_imgs_with_num_blobs = 0

        for idx, data in enumerate(imgs_data):

            if data["label"][num_blobs] == 1: # data is for an image with num_blobs blobs
                num_imgs_with_num_blobs += 1
                
                max_glimpse = 2
                min_glimpse = 0
                glimpses = 5#max_glimpse - min_glimpse + 1
                
                for i in [0, 1, 2, 3, 4]:#range(min_glimpse, max_glimpse + 1):
                    # Histogram of softmaxes
                    new_hist += data["classifications"][i][0] / glimpses
                    
                    # Histogram of choices
                    choice = np.argmax(data["classifications"][i][0])
                    choice_list = [0] * z_size
                    choice_list[choice] = 1 / glimpses
                    choice_hist += choice_list
                    
#                 glimpse = 1
        
#                 new_hist += data["classifications"][glimpse][0]
        
#                 choice = np.argmax(data["classifications"][glimpse][0])
#                 choice_list = [0] * z_size
#                 choice_list[choice] = 1
#                 choice_hist += choice_list
                
        print("num_imgs_with_num_blobs: ", num_imgs_with_num_blobs)
        
        new_hist = new_hist / num_imgs_with_num_blobs
        print("new_hist: ", new_hist)
        choice_hist = choice_hist / num_imgs_with_num_blobs

        x = np.linspace(-2, 11.0, 1000)
        source = ColumnDataSource(data=dict(color=["red"] * z_size, top=new_hist, bottom=np.zeros(z_size), left=np.arange(z_size) + m - 0.55, right=np.arange(1, z_size + 1) - m - 0.55))
        source2 = ColumnDataSource(data=dict(color=["yellow"] * z_size, top=choice_hist, bottom=np.zeros(z_size), left=np.arange(z_size) + m - 0.45, right=np.arange(1, z_size + 1) - m - 0.45))
        source.data["color"][num_blobs] = "lime"
        p1.quad('left', 'right', 'top', 'bottom', source=source, color="color", alpha=1)
        p1.quad('left', 'right', 'top', 'bottom', source=source2, color="color", alpha=0.5)


        # FORMAT PLOT ##############################

        p1.xaxis.axis_label = 'Number of Blobs'
        p1.yaxis.axis_label = 'Classification Probability'
        p1.xaxis[0].ticker=FixedTicker(ticks=np.arange(z_size))
        
        
        # PLOT CURVES #############################
        
        # Find the mean
        for j in range(z_size):
            values_sum += j * new_hist[j]
        mu = values_sum
        print("mu: ", mu)

        # Find the standard deviation
        for k in range(z_size):
            sqr_sum += new_hist[k] * ((k - mu) ** 2)
        sigma = np.sqrt(sqr_sum)
        
        curves.append((sigma * 2, x, mu, p2, str(num_blobs),
                       "#" + str(randint(2, 9)) + str(randint(2, 9))+ str(randint(2, 9))+ str(randint(2, 9))+ str(randint(2, 9))+ str(randint(2, 9))))
        
        curve(sigma, x, mu, p1) # plot gaussian curve
#         a = np.argmax(new_hist)
#         skew_curve(sigma, x, mu, a, p1)
#         combined_curve(sigma, x, mu, a, p1)

        show(gridplot(p1, ncols=2, plot_width=500, plot_height=400, toolbar_location=None))
    
    for stats in curves:
        curve(*stats)
    show(p2)
    

def on_click2(b2, new_image=True):
    """Load new random image after button is clicked."""
    
    b2.description = "Loading..."
    update_curves()
    b2.description = "Next (Random) Image"

b2.on_click(on_click2)


def on_change2(change):
    """Change the iteration number to new dropdown selection."""
    
    if change['type'] == 'change' and change['name'] == 'value':
        on_click(b2, new_image=False)
        

dropdown2.observe(on_change2)
display(HBox([b2, dropdown2]))

In [ ]:
edges # the left and right edges of the bars

In [ ]:
hist # the heights of the bars

In [ ]:
len(data["classifications"]) # glimpses

In [ ]:
len(data["classifications"][0]) # machine and human (2)

In [ ]:
len(data["classifications"][0][0])

In [ ]:
3 ** 2

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

from scipy import linspace
from scipy import pi,sqrt,exp
from scipy.special import erf

def pdf(x):
    return 1/sqrt(2*pi) * exp(-x**2/2)

def cdf(x):
    return (1 + erf(x/sqrt(2))) / 2

def skew(x,e=0,w=1,a=0):
    t = (x-e) / w
    return 2 / w * pdf(t) * cdf(a*t)
    # You can of course use the scipy.stats.norm versions
    # return 2 * norm.pdf(t) * norm.cdf(a*t)


n = 2**10

e = 1.0 # location
w = 2.0 # scale

x = linspace(-10,10,n) 

for a in range(-3,4):
    p = skew(x,e,w,a)
    plt.plot(x,p)
    plt.show()

show()

In [ ]:
from analysis import count_blobs, random_count_image
print("Done!")

In [ ]:
print(count_blobs(100000, True))